In [ ]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive')

  # Install the latest version of torchtext library quietly without showing output

  !pip install torchinfo -qq

  basepath = '/content/drive/MyDrive/HACKATHON_2025'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Step 1: Load the datasets
df = pd.read_csv("/content/drive/MyDrive/HACKATHON_2025/Disaster_Risk_Dataset_onlyimpcolumns.csv")
cbsa_df = pd.read_csv("/content/drive/MyDrive/HACKATHON_2025/cbsa2fipsxw.csv")

# Step 2: Filter for Metropolitan Statistical Areas only
msa_df = cbsa_df[cbsa_df["metropolitanmicropolitanstatis"] == "Metropolitan Statistical Area"]

# Step 3: Create MSA mapping
msa_mapping = {
    (str(row["statename"]).strip(), str(row["countycountyequivalent"]).strip()): f"310M700US{str(row['cbsacode']).zfill(5)}"
    for _, row in msa_df.iterrows()
}

# Step 4: Normalize state and county columns
df["STATE"] = df["STATE"].astype(str).str.strip()
df["COUNTY"] = df["COUNTY"].astype(str).str.strip()

# Step 5: Assign GEO_ID using the mapping
df["GEO_ID"] = df.apply(
    lambda row: msa_mapping.get((row["STATE"], row["COUNTY"])), axis=1
)

# Step 6: Remove rows where GEO_ID is NaN (i.e., no MSA match)
df = df.dropna(subset=["GEO_ID"])

# Step 7: Reorder columns with GEO_ID first
cols = ["GEO_ID"] + [col for col in df.columns if col != "GEO_ID"]
df = df[cols]

# Step 8: Save the final filtered DataFrame as CSV
output_path = "/content/drive/MyDrive/HACKATHON_2025/water_risk_with_msa_geo_id.csv"
df.to_csv(output_path, index=False)

# Optional: Print preview and stats
print(df.head())
print(f"Final dataset saved to: {output_path}")
print(f"Number of rows in final dataset: {len(df)}")


           GEO_ID  Unnamed: 0    STATE   COUNTY    WNTW_RISKV      WNTW_RISKR  \
0  310M700US33860           0  Alabama  Autauga   8494.906508        Very Low   
1  310M700US19300           1  Alabama  Baldwin  65619.701638  Relatively Low   
3  310M700US13820           3  Alabama     Bibb   7496.186940        Very Low   
4  310M700US13820           4  Alabama   Blount  17175.160729        Very Low   
7  310M700US11500           7  Alabama  Calhoun  35926.190293  Relatively Low   

   WNTW_EALS     WNTW_ALRB     WNTW_ALRP     WNTW_ALRA  WNTW_ALR_NPCTL  
0  15.784587  2.687716e-07  7.410082e-09  8.725777e-06       10.461158  
1  56.205509  1.268231e-09  2.287120e-08  1.548360e-07       13.339523  
3  13.308573  9.014679e-07  1.270300e-08  1.202015e-05       16.991643  
4  23.645930  5.268425e-07  1.482016e-08  2.002965e-07       12.039616  
7  39.306716  2.387269e-07  1.799591e-08  6.346198e-07       11.451563  
Final dataset saved to: /content/drive/MyDrive/HACKATHON_2025/water_risk_wi